In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import json

with open("/home/jovyan/work/.credentials.json", "r") as credential:
    credential = json.load(credential)
    url = credential["url"]

def get_days(row):
    """return days of drug exposure after condition_start_date"""
    condition_start_date = row["condition_start_date"]
    drug_exposure_start_date = row["drug_exposure_start_date"]
    drug_exposure_end_date = row["drug_exposure_end_date"]
    
    if condition_start_date <= drug_exposure_start_date:
        return drug_exposure_end_date - drug_exposure_start_date + timedelta(days=1)
    else:
        if condition_start_date <= drug_exposure_end_date:
            return drug_exposure_end_date - condition_start_date + timedelta(days=1)
        else:
            return timedelta(days=0)

In [2]:
# DB 연결
engine = create_engine(url, connect_args={'options': '-csearch_path={}'.format('de')})
df_person = pd.read_sql_table(
    "person",
    con=engine
)

df_condition = pd.read_sql_table(
    "condition_occurrence",
    con=engine
)

df_drug = pd.read_sql_table(
    "drug_exposure",
    con=engine
)

In [3]:
# 제 2형 당뇨병 정보만 가져오기
diabetes2_concept_id = [3191208, 36684827, 3194332, 3193274, 43531010,
                        4130162, 45766052, 45757474, 4099651, 4129519,
                        4063043, 4230254, 4193704, 4304377, 201826,
                        3194082, 3192767]

df_diabetes2 = df_condition[df_condition["condition_concept_id"].isin(diabetes2_concept_id)]

In [4]:
# 제2형 당뇨병 환자들의 출생 정보 가져오기
df_merged1 = pd.merge(df_diabetes2,
                      df_person,
                      how="left",
                      left_on="person_id",
                      right_on="person_id")

# 제 2형 당뇨병이면서, condition occurence date 당시 18세 이상인 사람들만 가져오기
over_18 = df_merged1["condition_start_datetime"] - df_merged1["birth_datetime"] > timedelta(days=18*365)
df_merged1 = df_merged1[over_18]

In [5]:
# Metformin 복용자의 person_id, drug_exposure_start_date, drug_exposure_end_date 가져오기
df_metformin = df_drug.loc[df_drug.drug_concept_id == 40163924] \
    [["person_id", "drug_exposure_start_date", "drug_exposure_end_date"]]

In [6]:
"""
제 2형 당뇨병 & 18세 이상인 사람들의 Metformin 복용 정보 가져오기
위의 조건을 모두 만족해야하므로 "inner" join
"""
df_merged2 = pd.merge(df_merged1,
                      df_metformin,
                      how="inner",
                      left_on="person_id",
                      right_on="person_id"
                     )

In [7]:
# 진단(당뇨병) 이후 Metformin을 복용한 일 수 구하기
df_merged2["exposure_days"] = df_merged2.apply(lambda row: get_days(row), axis=1)

In [8]:
# 환자 별 Metformin 전체 투약 일 수 산출
df_exposure = df_merged2.groupby("person_id")["exposure_days"] \
    .sum()\
    .to_frame()\
    .reset_index()

# 전체 투약 일수가 90일 이상인 경우 산출
res = len(df_exposure.loc[df_exposure.exposure_days >= timedelta(days=90)])

print(f'제 2형 당뇨병이며, 만 18세 이상인 환자 중 Metformin을 90일 이상 복용한 환자수: {res}명')

제 2형 당뇨병이며, 만 18세 이상인 환자 중 Metformin을 90일 이상 복용한 환자수: 30명
